In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from latentis.pipeline.flow import NNPipeline, Flow


Procrustes = NNPipeline(name="Procrustes")

In [ ]:
Procrustes.add(
    Flow(name="fit", inputs=["fit_x", "fit_y"], outputs="translator")
    .add(block="x_scaler", method="fit_transform", inputs="fit_x:x", outputs="scaled_x")
    .add(block="y_scaler", method="fit_transform", inputs="fit_y:x", outputs="scaled_y")
    # .add_string("padding.fit_transform(scaled_x:x, scaled_y:y) -> padded_x, padded_y")
    .add(
        block="padding",
        method="fit_transform",
        inputs=["scaled_x:x", "scaled_y:y"],
        outputs=["padded_x", "padded_y"],
    )
    .add(
        block="estimator",
        method="fit",
        inputs=["padded_x:x", "padded_y:y"],
        outputs="translator",
    )
)
Procrustes.flows["fit"].to_pydot()

In [ ]:
Procrustes.add(
    Flow(name="transform", inputs="x", outputs="x_transformed")
    .add(block="x_scaler", method="transform", inputs="x", outputs="scaled_x")
    .add(
        block="padding",
        method="transform",
        inputs="scaled_x:x",
        outputs=["padded_x"],
    )
    .add(
        block="estimator",
        method="transform",
        inputs="padded_x:x",
        outputs="translated_x",
    )
    .add(
        block="y_scaler",
        method="inverse_transform",
        inputs="translated_x:x",
        outputs="x_transformed",
    )
)
Procrustes.flows["transform"].to_pydot()

In [ ]:
from latentis.transform.base import StandardScaling
from latentis.transform.dim_matcher import ZeroPadding
from latentis.transform.translate.aligner import MatrixAligner
from latentis.transform.translate.functional import svd_align_state

In [ ]:
Procrustes.build(
    x_scaler=StandardScaling(),
    y_scaler=StandardScaling(),
    padding=ZeroPadding(),
    estimator=MatrixAligner(name="svd", align_fn_state=svd_align_state),
)

In [ ]:
import torch

In [ ]:
x = torch.randn(5, 4)
y = torch.randn(5, 4)

In [ ]:
Procrustes.run(flow="fit", fit_x=x, fit_y=y)

In [ ]:
Procrustes.run(flow="transform", x=x)

In [ ]:
from latentis.transform.base import MeanLPNorm
from latentis.transform import Identity

In [ ]:
x = torch.randn(5, 4)
anchors = x[:2]

In [ ]:
RelProj = NNPipeline(name="RelProj")

In [ ]:
RelProj.add(
    Flow(name="fit", inputs="anchors", outputs="rel_anchors")
    .add(
        block="abs_transform",
        method="fit_transform",
        inputs="anchors:x",
        outputs="abs_anchors",
    )
    .add(
        block="projection",
        method="fit",
        inputs="abs_anchors:anchors",
        outputs="rel_proj",
    )
    .add(
        block="projection",
        method="transform",
        inputs="abs_anchors:x",
        outputs="rel_anchors",
    )
    .add(
        block="rel_transform",
        method="fit_transform",
        inputs="rel_anchors:x",
        outputs="rel_anchors",
    )
)
RelProj.flows["fit"].to_pydot()

In [ ]:
RelProj.add(
    Flow(name="transform", inputs="x", outputs="rel_x")
    .add(block="abs_transform", method="transform", inputs="x", outputs="abs_x")
    .add(block="projection", method="transform", inputs=["abs_x:x"], outputs="rel_x")
    .add(block="rel_transform", method="transform", inputs="rel_x:x", outputs="rel_x")
)
RelProj.flows["transform"].to_pydot()

In [ ]:
from latentis.transform import projection
from latentis.transform.projection import RelativeProjection


RelProj = RelProj.build(
    abs_transform=MeanLPNorm(p=2),
    projection=RelativeProjection(projection_fn=projection.cosine_proj),
    rel_transform=Identity(),
)
RelProj

In [ ]:
RelProj.run(flow="fit", anchors=anchors)

In [ ]:
RelProj.run(flow="transform", x=x)

In [ ]:
translator = NNPipeline(
    name="translator",
    flows={
        "fit": Flow(name="fit", inputs=["fit_x", "fit_y"], outputs="estimator")
        .add(block="x_transform", method="fit_transform", inputs="fit_x:x", outputs="x")
        .add(block="y_transform", method="fit_transform", inputs="fit_y:x", outputs="y")
        .add(
            block="padder",
            method="fit_transform",
            inputs=["x", "y"],
            outputs=["padded_x", "padded_y"],
        )
        .add(
            block="aligner",
            method="fit",
            inputs=["padded_x:x", "padded_y:y"],
            outputs=["aligner_state"],
        ),
        #
        "transform": Flow(name="transform", inputs=["x"], outputs=["translated_x"])
        .add(block="x_transform", method="transform", inputs="x", outputs="x")
        .add(block="padder", method="transform", inputs="x", outputs="x")
        .add(block="aligner", method="transform", inputs="x", outputs="x")
        .add(block="padder", method="inverse_transform", inputs="x:y", outputs="x")
        .add(
            block="y_transform",
            method="inverse_transform",
            inputs="x",
            outputs="translated_x",
        ),
    },
)

In [ ]:
translator.flows["fit"].to_pydot()

In [ ]:
translator.flows["transform"].to_pydot()